In [1]:
# Dependencies
# ------------------------
import os
import pandas as pd
import numpy as numpy
from math import sin, cos, sqrt, atan2, radians
# from sqlalchemy import create_engine
# database_path= "../Resources/getOutside_data.sqlite"
# from sqlalchemy.orm import Session
# from sqlalchemy import create_engine, func, inspect, MetaData, Table

import json 
#get gasStations geojson
with open("../Resources/chargerStations.js") as f:
    chargerStationsData = json.load(f)
#     print (chargerStationsData.keys())
with open("../Resources/gasStations.js") as f:
    gasStationsData = json.load(f)
#     print (gasStationsData.keys())

In [2]:
import pymongo

In [3]:
# #Grab features key and find the long and lat for Gas Stations

# # gas_long_lat=gasStationsData["features"][0]["geometry"]
# # gas_long_lat

# gas_long=gasStationsData["features"][1]["geometry"]["coordinates"][0]
# gas_long
# gas_lat=gasStationsData["features"][1]["geometry"]["coordinates"][1]
# gas_lat
# print(gas_long, gas_lat)

In [4]:
# # Find the gas stations amenity name
# gasStations=gasStationsData["features"][0]["properties"]["amenity"]
# gasStations

In [5]:
# # Iterate through the features key and grab all the amenity, long, and lat information for gas stations
# for gas_stations in gasStationsData["features"]:
#     gas_station=gas_stations["properties"]["amenity"]
#     print(gas_station)
#     gas_long=gas_stations["geometry"]["coordinates"][0]
#     gas_lat=gas_stations["geometry"]["coordinates"][1]
#     print(gas_long, gas_lat)
#     # write to mongo

### Begin camping_14ers_db

In [6]:
# Create Pandas df using flat csv camp ground file
file_path = "../Resources/southwest_camp_data.csv"
camps = pd.read_csv(file_path,encoding="utf-8")
# camps.head()

In [7]:
# Locate columns for information and grab only CO information
camp = camps[["Long","Lat", "name", "type","phone", "dates", "# sites", "Elevation","State"]]
co_camp = camp[camp["State"] == "CO"]
co_example = co_camp
co_example

,Long,Lat,name,type,phone,dates,# sites,Elevation,State
259,-106.345,37.379,Alamosa,NF,719.274.8971/719-852-5941,,10,8665,CO
260,-106.858,38.654,Almont,NF,970.641.0471/970.874.6600,mid may-mid sep,10,8018,CO
261,-105.562,38.080,Alvarado,NF,719.269.8500/719.545.8737,early jun-mid oct,50,8986,CO
262,-107.665,38.019,Amphitheater,NF,970.240.5300/970.874.6600,mid jun-late sep,35,8320,CO
263,-106.221,38.583,Angel of Shavano,NF,719.539.3591/719.545.8737,,20,9180,CO
...,...,...,...,...,...,...,...,...,...
761,-108.055,37.885,Woods Lake,NF,970.327.4261/970.874.6600,mid may-mid oct,41,9455,CO
762,-107.287,37.973,Wupperman,CP,970.944.2319,,,9018,CO
763,-104.940,38.737,Wye,NF,719.636.1602/719.545.8737,mid may-mid sep,21,9790,CO
764,-106.678,39.502,Yeoman Park,NF,970.328.6388/970.945.2521,,24,9081,CO


In [8]:
# Create Pandas Df using flat csv 14ers data
file_path2 = "../Resources/14ers_data.csv"
mountains = pd.read_csv(file_path2, encoding = "ISO-8859–1")
mountains.head()

,ID,Mountain Peak,Mountain Range,Elevation_ft,fourteener,Prominence_ft,Isolation_mi,Peak Longitude,Peak Latitude,Standard Route,Standard Route Distance (miles),Elevation Gain_ft,Difficulty,Traffic Low,Traffic High,photo
0,1,Mt. Elbert,Sawatch Range,14440,Y,9093,670.00,-106.4454,39.1178,Northeast Ridge,9.50,4700,Class 1,20000,25000,https://www.14ers.com/photos/mtelbert/peakphot...
1,2,Mt. Massive,Sawatch Range,14428,Y,1961,5.06,-106.4757,39.1875,East Slopes,14.50,4500,Class 2,7000,10000,https://www.14ers.com/photos/mtmassive/peakpho...
2,3,Mt. Harvard,Sawatch Range,14421,Y,2360,14.93,-106.3207,38.9244,South Slopes,14.00,4600,Class 2,5000,7000,https://www.14ers.com/photos/harvardgroup/peak...
3,4,Blanca Peak,Sangre de Cristo Range,14351,Y,5326,103.40,-105.4856,37.5775,Northwest Ridge,17.00,6500,Hard Class 2,1000,3000,https://www.14ers.com/photos/blancagroup/peakp...
4,5,La Plata Peak,Sawatch Range,14343,Y,1836,6.28,-106.4729,39.0294,Northwest Ridge,9.25,4500,Class 2,5000,7000,https://www.14ers.com/photos/laplatapeak/peakp...


In [9]:
# Create empty list to append information
distance_list = []
mountain_list = []
camp_list = []

#iterate through mountains and co_example append results to empty list. use math to find radius of 20 miles
for l,m,n in zip(mountains["Mountain Peak"],mountains["Peak Latitude"],mountains["Peak Longitude"]):
    lat1 = radians(m)
    lon1 = radians(n)
    for i, j, k in zip(co_example["Lat"],co_example["Long"], co_example["name"]):
        lat2 = radians(i)
        lon2 = radians(j)
        camp_name = k
        R = 3958.8
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = (R * c)
        if distance <= 20:
            mountain_list.append(l)
            distance_list.append(distance)
            camp_list.append(camp_name)
            #write to mongo

In [10]:
# Create Pandas DF and utilizing the lists above. Drop any duplicates from the new DF
distance_df = pd.DataFrame({
    "Mountain Peak": mountain_list,
    "Distance from Campsite (miles)" : distance_list,
    "name": camp_list})
distance_df = distance_df.drop_duplicates()
distance_df

,Mountain Peak,Distance from Campsite (miles),name
0,Mt. Elbert,11.667981,Baby Doe
1,Mt. Elbert,11.652809,Belle of Colorado
2,Mt. Elbert,16.725207,Chapman
3,Mt. Elbert,6.697035,Dexter
4,Mt. Elbert,17.685043,Difficult
...,...,...,...
1059,North Maroon Peak,15.578815,Redstone
1060,North Maroon Peak,7.913072,Silver Bar
1061,North Maroon Peak,6.766475,Silver Bell
1062,North Maroon Peak,5.933690,Silver Queen


In [11]:
# Merge the distance_df and the co_example information on name. Second merge for updated camp_merge and mountains. 
# Merge on Mountain Peak then drop any duplicates from the mountains df and rename to new clean_mountains.
camp_merge = pd.merge(distance_df, co_example[["Long", "Lat", "name"]], on ="name")
mountains=pd.merge(camp_merge,mountains[["Mountain Peak", "Peak Longitude", "Peak Latitude"]], on ="Mountain Peak")
clean_mountains=mountains.drop_duplicates()
clean_mountains
#write to mongo

,Mountain Peak,Distance from Campsite (miles),name,Long,Lat,Peak Longitude,Peak Latitude
0,Mt. Elbert,11.667981,Baby Doe,-106.351,39.270,-106.4454,39.1178
1,Mt. Elbert,11.652809,Belle of Colorado,-106.349,39.269,-106.4454,39.1178
2,Mt. Elbert,16.725207,Chapman,-106.637,39.309,-106.4454,39.1178
3,Mt. Elbert,6.697035,Dexter,-106.325,39.092,-106.4454,39.1178
4,Mt. Elbert,17.685043,Difficult,-106.774,39.141,-106.4454,39.1178
...,...,...,...,...,...,...,...
1098,Culebra Peak,14.263277,Bear Lake,-105.143,37.326,-105.1858,37.1224
1099,Culebra Peak,14.263277,Bear Lake,-107.070,40.045,-105.1858,37.1224
1100,Culebra Peak,13.418729,Blue Lake,-105.139,37.313,-105.1858,37.1224
1101,Culebra Peak,9.621643,Monument Lake Park,-105.047,37.207,-105.1858,37.1224


In [12]:
#Write to csv
clean_mountains.to_csv(r'../Resources/camping_14ers.csv')

In [13]:
# Declare the collection for camping & 14ers records
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.classDB

camping_14ers = db.camping_14ers_db
data = clean_mountains.to_dict(orient = "records")
camping_14ers.insert_many(data)

### Begin charging_db_LongLat

In [14]:
# #chargerStationsData is defined where it was loaded as a json object
# #view features key within chargerStationsData
# chargerStationsData["features"]

In [15]:
# print (chargerStationsData.keys())

In [16]:
# Grab features key and find teh long and lat for Charging Stations

# stations_long_lat=chargerStationsData["features"][0]["geometry"]
# stations_long_lat
charger_long=chargerStationsData["features"][1]["geometry"]["coordinates"][0]
# charger_long
charger_lat=chargerStationsData["features"][1]["geometry"]["coordinates"][1]
# charger_lat
print(charger_long, charger_lat)

-107.88321508498254 37.27231589554491


In [17]:
# Find the charging stations amenity name 
chargerName=chargerStationsData["features"][0]["properties"]["amenity"]
chargerName

'charging_station'

In [18]:
# Iterate through the features key and grab all the amenity, long, and lat information for charging stations
# for charger_stations in chargerStationsData["features"]:
#     charger_station=charger_stations["properties"]["amenity"]
# #     print(charger_station)
#     charger_long=charger_stations["geometry"]["coordinates"][0]
#     charger_lat=charger_stations["geometry"]["coordinates"][1]
# #     print(charger_long, charger_lat)

# write to mongo

In [19]:
# Create empty list to append data to
# Iterate through the chargerStationsData append stations, long, and lat.
charger_stations=[]
charger_longs=[]
charger_lats=[]
for row in chargerStationsData["features"]:
    charger_station= row['properties']['amenity']
    
#     charger_station = charger_stations["properties"]["amenity"]
# #     print(charger_station)

    charger_long = row["geometry"]["coordinates"][0]
    charger_lat = row["geometry"]["coordinates"][1]
    charger_longs.append(charger_long)
    charger_lats.append(charger_lat)
    charger_stations.append(charger_station)
#     print(charger_long, charger_lat)

In [20]:
# # Create empty list to append data to 
# # Iterate through the chargerStationsData append stations, long, and lat.
# # Gives error b/c you are trying to use charger_stations as both a list to append data to as well as in the for loop
# #adjusted for loop above to row and that did it. Append to charger_df below 
# charger_stations=[]
# charger_longs=[]
# charger_lats=[]
# for charger_station in chargerStationsData["features"]:
#     charger_station = charger_stations["properties"]["amenity"]
# #     print(charger_station)
#     charger_stations.append(charger_station)
#     charger_long = charger_stations["geometry"]["coordinates"][0]
#     charger_lat = charger_stations["geometry"]["coordinates"][1]
#     charger_longs.append(charger_long)
#     charger_lats.append(charger_lat)
# #     print(charger_long, charger_lat)

In [21]:
# Create new DF with charger information.
charger_df=pd.DataFrame({
    "Charger Longitude": charger_longs,
    "Charger Latitude": charger_lats,
    "Charger Stations": charger_stations
})
# charger_df
# DO NOT NEED, FOR LOOP ABOVE WORKING
# charger_df["Name"]="charging_station"
charger_df

,Charger Longitude,Charger Latitude,Charger Stations
0,-108.288210,37.345423,charging_station
1,-107.883215,37.272316,charging_station
2,-108.549134,39.055387,charging_station
3,-108.604298,39.090749,charging_station
4,-108.568484,39.067964,charging_station
...,...,...,...
140,-105.063225,40.405828,charging_station
141,-105.007776,40.408870,charging_station
142,-104.991169,40.441451,charging_station
143,-105.094787,40.521416,charging_station


In [22]:
#Write to csv
charger_df.to_csv(r'../Resources/charging_db_LongLat.csv')

In [23]:
# Declare the collection for charger stations to get long and lat
charger_LongLat = db.charging_db_LongLat
data = charger_df.to_dict(orient = "records")
charger_LongLat.insert_many(data)

### Begin charging_14ers_db

In [24]:
# Create empty list to append charger distance and mountain distance within 20 miles of each. 
charger_distance_list = []
mountain_charger_list = []
for l,m,n in zip(mountains["Mountain Peak"],mountains["Peak Latitude"],mountains["Peak Longitude"]):
    lat1 = radians(m)
    lon1 = radians(n)
    for i, j, k in zip(charger_df["Charger Latitude"], charger_df["Charger Longitude"], charger_df["Charger Stations"]):
        lat2 = radians(i)
        lon2 = radians(j)
        Name = k
        R = 3958.8
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = (R * c)
        if distance <= 20:
            mountain_charger_list.append(l)
            charger_distance_list.append(distance)

In [25]:
# Create a new DF and display the charging distance and mountain peak above 
charging_df=pd.DataFrame({
    "Charging Distance" : charger_distance_list,
    "Mountain Peak" : mountain_charger_list
    })
charging_df

,Charging Distance,Mountain Peak
0,18.320254,Mt. Massive
1,18.320254,Mt. Massive
2,18.320254,Mt. Massive
3,18.320254,Mt. Massive
4,18.320254,Mt. Massive
...,...,...
1970,14.911363,Pikes Peak
1971,17.810433,Pikes Peak
1972,11.817678,Pikes Peak
1973,14.911363,Pikes Peak


In [26]:
#merge charging_df to mountains df on Mountain Peak
charger_merge=pd.merge(charging_df, mountains[["Mountain Peak", "Peak Longitude", "Peak Latitude"]], on ="Mountain Peak")
clean_charger_merge=charger_merge.drop_duplicates()
clean_charger_merge

,Charging Distance,Mountain Peak,Peak Longitude,Peak Latitude
0,18.320254,Mt. Massive,-106.475700,39.187500
841,19.462486,Mt. Lincoln,-106.111600,39.351500
1930,16.286941,Quandary Peak,-106.106400,39.397300
3086,13.456950,Mt. of the Holy Cross,-106.481700,39.466800
3711,19.804094,Mt. Cameron,-106.118501,39.347165
...,...,...,...,...
43957,17.966456,Mt. Bierstadt,-105.668800,39.582600
45182,10.216458,Longs Peak,-105.615100,40.255000
45711,11.817678,Pikes Peak,-105.044200,38.840500
45727,14.911363,Pikes Peak,-105.044200,38.840500


In [27]:
#Write to csv
clean_charger_merge.to_csv(r'../Resources/charging_14ers_db.csv')

In [28]:
# Declare the collection for charger stations with 14ers Peak information
charger_14ers = db.charging_14ers_db
data = clean_charger_merge.to_dict(orient = "records")
charger_14ers.insert_many(data)

### Begin gas_db_LongLat

In [29]:
# #chargerStationsData is defined where it was loaded as a json object
# #view features key within chargerStationsData
gasStationsData["features"]

[{'type': 'Feature',
  'properties': {'gid': 3164016902,
   'access': None,
   'addr:housename': None,
   'addr:housenumber': None,
   'amenity': 'fuel',
   'brand': None,
   'building': None,
   'covered': None,
   'ele': None,
   'name': None,
   'operator': None,
   'shop': None,
   'tid': None,
   'territory_name': None},
  'geometry': {'type': 'Point',
   'coordinates': [-108.67551598487225, 37.21848989554922]}},
 {'type': 'Feature',
  'properties': {'gid': 957771611,
   'access': None,
   'addr:housename': None,
   'addr:housenumber': None,
   'amenity': 'fuel',
   'brand': None,
   'building': None,
   'covered': None,
   'ele': None,
   'name': 'Texaco',
   'operator': None,
   'shop': None,
   'tid': None,
   'territory_name': None},
  'geometry': {'type': 'Point',
   'coordinates': [-108.58960288488421, 37.3488424955388]}},
 {'type': 'Feature',
  'properties': {'gid': 7691925828,
   'access': None,
   'addr:housename': None,
   'addr:housenumber': None,
   'amenity': 'fuel',


In [30]:
# print (gasStationsData.keys())

In [31]:
# Grab features key and find the long and lat for Gas Stations

gas_long_lat=gasStationsData["features"][0]["geometry"]
# gas_long_lat

gas_long=gasStationsData["features"][1]["geometry"]["coordinates"][0]
# gas_long
gas_lat=gasStationsData["features"][1]["geometry"]["coordinates"][1]
# gas_lat
print(gas_long, gas_lat)

-108.58960288488421 37.3488424955388


In [32]:
# Find the gas stations amenity name
gasStations=gasStationsData["features"][0]["properties"]["amenity"]
# gasStations

In [33]:
# Iterate through the features key and grab all the amenity, long, and lat information for gas stations
for gas_stations in gasStationsData["features"]:
    gas_station=gas_stations["properties"]["amenity"]
    print(gas_station)
    gas_long=gas_stations["geometry"]["coordinates"][0]
    gas_lat=gas_stations["geometry"]["coordinates"][1]
    print(gas_long, gas_lat)
    # write to mongo

fuel
-108.67551598487225 37.21848989554922
fuel
-108.58960288488421 37.3488424955388
fuel
-108.53716468489151 37.46381209552963
fuel
-108.91180168483935 37.76536409550584
fuel
-108.9130120848392 37.765746295505814
fuel
-107.85873758498595 37.226794895548544
fuel
-107.86126098498559 37.22976989554832
fuel
-107.63337438501732 37.11696909555737
fuel
-107.60001048502194 37.23020249554827
fuel
-107.59418478502276 37.23295739554806
fuel
-107.5866745850238 37.23397389554798
fuel
-107.18345658507992 37.233288695548026
fuel
-107.07863018509455 37.25459059554631
fuel
-107.07781218509464 37.254727195546316
fuel
-107.07511648509501 37.25630059554619
fuel
-107.8840591849824 37.271005095545
fuel
-107.87759158498332 37.28199429554413
fuel
-107.8776059849833 37.283627495544
fuel
-108.03178438496188 37.688094495511905
fuel
-107.05247138509819 37.268415995545205
fuel
-107.01543548510332 37.26669989554535
fuel
-106.99773668510579 37.26915639554517
fuel
-106.99709578510588 37.26970449554512
fuel
-107.1946

-105.23296138535143 39.95388689534212
fuel
-105.2757616853455 40.016305195337694
fuel
-105.26336008534723 40.01444799533784
fuel
-105.25159678534885 40.014359095337845
fuel
-105.25329068534862 40.015037295337784
fuel
-105.2635529853472 40.02367599533718
fuel
-105.25397358534853 40.028857695336804
fuel
-105.22571888535245 40.01422219533785
fuel
-105.22494508535256 40.01434959533785
fuel
-105.25889988534784 40.03613119533628
fuel
-105.17884128535898 40.014217195337864
fuel
-105.10941398536865 39.73073549535808
fuel
-105.0816315853725 39.745987095356995
fuel
-105.11004198536857 39.7695826953553
fuel
-105.10922288536868 39.76959969535529
fuel
-105.06292638537512 39.74741429535689
fuel
-105.05354478537642 39.73873739535751
fuel
-105.05363848537641 39.74012339535742
fuel
-105.07293638537372 39.76958469535528
fuel
-105.06212398537522 39.76968239535528
fuel
-105.02551458538034 39.74911299535676
fuel
-105.0437043853778 39.76918659535531
fuel
-105.10915868536867 39.778602595354634
fuel
-105.1030

In [34]:
# Create empty list to append data.
# Iterate through the features key and grab all the amenity, long, and lat information for gas stations
gas_stations=[] #Plural for 
gas_longs=[]
gas_lats=[]
for row in gasStationsData["features"]:
    gas_station=row["properties"]["amenity"]
#     print(gas_station)
    gas_long=row["geometry"]["coordinates"][0]
    gas_lat=row["geometry"]["coordinates"][1]
    
    gas_longs.append(gas_long)
    gas_lats.append(gas_lat)
    gas_stations.append(gas_station)

#     print(gas_long, gas_lat)

In [35]:
# Create new DF with gas information.
gas_df=pd.DataFrame({
    "Gas Longitude": gas_longs,
    "Gas Latitude": gas_lats,
    "Gas Stations": gas_stations
})
# charger_df
# DO NOT NEED, FOR LOOP ABOVE WORKING
# charger_df["Name"]="charging_station"
gas_df

,Gas Longitude,Gas Latitude,Gas Stations
0,-108.675516,37.218490,fuel
1,-108.589603,37.348842,fuel
2,-108.537165,37.463812,fuel
3,-108.911802,37.765364,fuel
4,-108.913012,37.765746,fuel
...,...,...,...
874,-103.167513,40.622376,fuel
875,-102.290899,39.702771,fuel
876,-102.302058,40.585749,fuel
877,-102.806008,40.821882,fuel


In [36]:
#Write to csv
gas_df.to_csv(r'../Resources/gas_db_LongLat.csv')

In [37]:
# Declare the collection for charger stations to get long and lat
gas_LongLat = db.gas_db_LongLat
data = gas_df.to_dict(orient = "records")
gas_LongLat.insert_many(data)

### Begin Gas Collection 2


In [38]:
# Create empty list to append charger distance and mountain distance within 20 miles of each. 
gas_distance_list = []
mountain_gas_list = []
for l,m,n in zip(mountains["Mountain Peak"],mountains["Peak Latitude"],mountains["Peak Longitude"]):
    lat1 = radians(m)
    lon1 = radians(n)
    for i, j, k in zip(gas_df["Gas Latitude"], gas_df["Gas Longitude"], gas_df["Gas Stations"]):
        lat2 = radians(i)
        lon2 = radians(j)
        Name = k
        R = 3958.8
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = (R * c)
        if distance <= 20:
            mountain_gas_list.append(l)
            gas_distance_list.append(distance)

In [39]:
#gas_distance_list

In [40]:
# Create a new DF to display gas distance and mountain peak lists above  
gas_filling_df=pd.DataFrame({
    "Gas Distance" : gas_distance_list,
    "Mountain Peak" : mountain_gas_list
    })
gas_filling_df

,Gas Distance,Mountain Peak
0,8.980248,Mt. Elbert
1,12.090677,Mt. Elbert
2,12.145856,Mt. Elbert
3,12.834498,Mt. Elbert
4,12.850290,Mt. Elbert
...,...,...
10564,15.845457,Pikes Peak
10565,16.087712,Pikes Peak
10566,18.009069,Pikes Peak
10567,18.699147,Pikes Peak


In [41]:
#merge gas_filling_df to mountains df on Mountain Peak
gas_merge=pd.merge(gas_filling_df, mountains[["Mountain Peak", "Peak Longitude", "Peak Latitude"]], on ="Mountain Peak")
clean_gas_merge=gas_merge.drop_duplicates()
clean_gas_merge

,Gas Distance,Mountain Peak,Peak Longitude,Peak Latitude
0,8.980248,Mt. Elbert,-106.4454,39.1178
29,12.090677,Mt. Elbert,-106.4454,39.1178
58,12.145856,Mt. Elbert,-106.4454,39.1178
87,12.834498,Mt. Elbert,-106.4454,39.1178
116,12.850290,Mt. Elbert,-106.4454,39.1178
...,...,...,...,...
284627,15.845457,Pikes Peak,-105.0442,38.8405
284643,16.087712,Pikes Peak,-105.0442,38.8405
284659,18.009069,Pikes Peak,-105.0442,38.8405
284675,18.699147,Pikes Peak,-105.0442,38.8405


In [42]:
#Write to csv
clean_gas_merge.to_csv(r'../Resources/gas_14ers_db.csv')

In [43]:
# Declare the collection for charger stations with 14ers Peak information
gas_14ers = db.gas_14ers_db
data = clean_gas_merge.to_dict(orient = "records")
gas_14ers.insert_many(data)

### Breweries

In [44]:
# Create Pandas df using flat csv camp ground file
file_path = "../Resources/cleaned_breweries.csv"
breweries = pd.read_csv(file_path,encoding="utf-8")
#breweries.head()

In [45]:
# Declare the collection for breweries to get long and lat
breweries_LongLat = db.breweries_db_LongLat
data = breweries.to_dict(orient = "records")
breweries_LongLat.insert_many(data)

In [46]:
# Create empty list to append brewery distance and mountain distance within 20 miles of each. 
brewery_distance_list = []
mountain_brewery_list = []
brewery_name_list=[]
for l,m,n in zip(mountains["Mountain Peak"],mountains["Peak Latitude"],mountains["Peak Longitude"]):
    lat1 = radians(m)
    lon1 = radians(n)
    for i, j, k in zip(breweries["Latitudes"], breweries["Longitudes"], breweries["Brewery"]):
        lat2 = radians(i)
        lon2 = radians(j)
        Brewery = k
        R = 3958.8
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))
        distance = (R * c)
        if distance <= 20:
            mountain_brewery_list.append(l)
            brewery_distance_list.append(distance)
            brewery_name_list.append(Brewery)

In [47]:
# Create a new DF to display brewery distance and mountain peak lists above  
brewery_df=pd.DataFrame({
    "Brewery Distance" : brewery_distance_list,
    "Mountain Peak" : mountain_brewery_list,
    "Brewery Name":brewery_name_list
    })
brewery_df

,Brewery Distance,Mountain Peak,Brewery Name
0,12.059106,Mt. Elbert,Two Mile Brewing Co
1,12.059106,Mt. Elbert,Two Mile Brewing Co
2,12.059106,Mt. Elbert,Two Mile Brewing Co
3,12.059106,Mt. Elbert,Two Mile Brewing Co
4,12.059106,Mt. Elbert,Two Mile Brewing Co
...,...,...,...
3797,17.655394,Pikes Peak,Nano 108 Brewing Co.
3798,11.808303,Pikes Peak,Phantom Canyon Brewing Co.
3799,12.729760,Pikes Peak,Storybook Brewing
3800,9.369500,Pikes Peak,Fossil Brewing Co.


In [48]:
#merge brewery_df to mountains df on Mountain Peak
brewery_merge=pd.merge(brewery_df, mountains[["Mountain Peak", "Peak Longitude", "Peak Latitude"]], on ="Mountain Peak")
clean_brewery_merge=brewery_merge.drop_duplicates()
clean_brewery_merge

,Brewery Distance,Mountain Peak,Brewery Name,Peak Longitude,Peak Latitude
0,12.059106,Mt. Elbert,Two Mile Brewing Co,-106.4454,39.1178
841,18.455272,Mt. Massive,Aspen Brewing Company,-106.4757,39.1875
870,10.658761,Mt. Massive,Two Mile Brewing Co,-106.4757,39.1875
2523,17.790090,La Plata Peak,Two Mile Brewing Co,-106.4729,39.0294
3484,16.446500,Mt. Lincoln,Outer Range Brewing Company,-106.1116,39.3515
...,...,...,...,...,...
98202,17.655394,Pikes Peak,Nano 108 Brewing Co.,-105.0442,38.8405
98218,11.808303,Pikes Peak,Phantom Canyon Brewing Co.,-105.0442,38.8405
98234,12.729760,Pikes Peak,Storybook Brewing,-105.0442,38.8405
98250,9.369500,Pikes Peak,Fossil Brewing Co.,-105.0442,38.8405


In [49]:
#Write to csv
clean_brewery_merge.to_csv(r'../Resources/brewery_14ers_db.csv')

In [50]:
# Declare the collection for breweries with 14ers Peak information
brewery_14ers = db.brewery_14ers_db
data = clean_brewery_merge.to_dict(orient = "records")
brewery_14ers.insert_many(data)

In [51]:
#merge distance dbs
distance_merge=pd.merge(clean_mountains, clean_charger_merge[["Mountain Peak", "Charging Distance"]], on ="Mountain Peak")
distance_merge=distance_merge.drop_duplicates()
distance_merge

,Mountain Peak,Distance from Campsite (miles),name,Long,Lat,Peak Longitude,Peak Latitude,Charging Distance
0,Mt. Massive,8.777135,Baby Doe,-106.351,39.270,-106.4757,39.1875,18.320254
1,Mt. Massive,8.814542,Belle of Colorado,-106.349,39.269,-106.4757,39.1875,18.320254
2,Mt. Massive,12.040109,Chapman,-106.637,39.309,-106.4757,39.1875,18.320254
3,Mt. Massive,10.428897,Dexter,-106.325,39.092,-106.4757,39.1875,18.320254
4,Mt. Massive,16.300096,Difficult,-106.774,39.141,-106.4757,39.1875,18.320254
...,...,...,...,...,...,...,...,...
1970,Pikes Peak,18.588746,Wildhorn,-105.253,39.055,-105.0442,38.8405,14.911363
1971,Pikes Peak,18.588746,Wildhorn,-105.253,39.055,-105.0442,38.8405,17.810433
1972,Pikes Peak,9.090248,Wye,-104.940,38.737,-105.0442,38.8405,11.817678
1973,Pikes Peak,9.090248,Wye,-104.940,38.737,-105.0442,38.8405,14.911363


In [52]:
### End for Now

In [53]:
# # Create Engine
# engine= create_engine(f"sqlite:///{database_path}")
# conn= engine.connect()

In [52]:
# type(conn)

In [53]:
# travel_dict=({ 'gas long':gas_long , 
#               'gas lat': gas_lat,
#               'gas stations': gas_stations
#               'charger long': charger_long,
#               'charger lat': charger_lat,
#               'charger stations': charger_stations,

             
#              })